# Mixture of Experts (MoE) 演示

本 Notebook 演示一个简单的 **稠密 MoE（Dense Mixture of Experts）** 层的实现与前向传播测试。

- 每个专家是一个线性层
- 门控网络决定每个输入对各专家的权重
- 输出是专家输出的加权和

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class Linear(nn.Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
    
    def forward(self, x):
        return self.fc(x)

In [ ]:
class MoELayer(nn.Module):
    def __init__(self, num_experts, in_features, out_features):
        super(MoELayer, self).__init__()
        self.num_experts = num_experts
        # 创建多个独立专家
        self.experts = nn.ModuleList([
            Linear(in_features, out_features) for _ in range(num_experts)
        ])
        # 门控网络：输入 -> 专家 logits
        self.gate = nn.Linear(in_features, num_experts)
    
    def forward(self, x):
        # x.shape: [batch_size, in_features]
        
        # 1. 门控打分 + softmax 归一化
        gate_logits = self.gate(x)  # [B, E]
        gate_score = F.softmax(gate_logits, dim=-1)  # [B, E]
        
        # 2. 所有专家并行计算输出
        expert_outputs = torch.stack([
            expert(x) for expert in self.experts
        ], dim=1)  # [B, E, out_features]
        
        # 3. 加权融合：batch matrix multiplication
        output = torch.bmm(
            gate_score.unsqueeze(1),   # [B, 1, E]
            expert_outputs             # [B, E, O]
        ).squeeze(1)  # [B, O]
        
        return output

In [ ]:
# 超参数设置
input_size = 5
output_size = 3
num_experts = 4
batch_size = 10

# 创建模型
model = MoELayer(num_experts, input_size, output_size)

# 随机输入
demo = torch.randn(batch_size, input_size)

# 前向传播
output = model(demo)

print("✅ 输入形状:", demo.shape)
print("✅ 输出形状:", output.shape)
assert output.shape == (batch_size, output_size), "输出形状不匹配！"

In [ ]:
# 可视化门控权重（帮助理解专家选择）
with torch.no_grad():
    gate_logits = model.gate(demo)
    gate_probs = F.softmax(gate_logits, dim=-1)
    
    print("🔍 第一个样本的专家权重分布:")
    for i, prob in enumerate(gate_probs[0]):
        print(f"  专家 {i}: {prob:.4f}")
    
    print(f"\n✅ 权重和（应为 1.0）: {gate_probs[0].sum().item():.6f}")

## 下一步建议

- 将此 MoE 层嵌入到完整模型中进行训练（如分类任务）
- 尝试实现 **稀疏 MoE（top-k gating）**
- 添加 **负载均衡辅助损失** 防止专家坍塌

欢迎继续探索！